In [129]:
import pandas as pd
from itertools import combinations


In [130]:
data = pd.read_excel('bread.xlsx', sheet_name='Sheet2')
data

,Transaction,Item
0,1,Bread
1,2,Scandinavian
2,3,Hot chocolate
3,3,Jam
4,3,Cookies
...,...,...
19635,9682,Coffee
19636,9682,Tea
19637,9683,Coffee
19638,9683,Pastry


In [131]:
dataset = {i: set() for i in data['Transaction'].unique()}
for i in range(len(data)):
    cur = data.iloc[i]
    dataset[cur['Transaction']].add(cur['Item'])

In [132]:
def apriori(data, min_support):
    L = data['Item'].unique()
    k = 1
    answer = {}
    while True:
        c = combinations(L, k)
        L = set()
        flag = True

        for now in c:
            cur_support = 0
            now = set(now)

            if 'NONE' in now:
                continue

            # support = number of transactions this set appears / total transactions
            for i in dataset:
                if now.issubset(dataset[i]):
                    cur_support += 1
            cur_support /= len(dataset)

            if cur_support >= min_support:
                L |= now
                answer[tuple(now)] = cur_support
                flag = False

        if flag:
            break

        k += 1
    return pd.DataFrame(answer.items(), columns=['Items', 'Support'])


In [133]:
frequent_itemset = apriori(data=data, min_support=0.04)
frequent_itemset

,Items,Support
0,"(Bread,)",0.324940
1,"(Hot chocolate,)",0.057916
2,"(Cookies,)",0.054034
3,"(Coffee,)",0.475081
4,"(Pastry,)",0.085510
5,"(Medialuna,)",0.061379
6,"(Tea,)",0.141643
7,"(Cake,)",0.103137
8,"(Sandwich,)",0.071346
9,"(Coffee, Pastry)",0.047214


In [134]:
def association(data, min_confidence):
    to_check = {}
    for i in range(len(data)):
        now = data.iloc[i]
        if len(now['Items']) > 1:
            to_check[now['Items']] = now['Support']

    for i in to_check:
        numerator = denominator1 = denominator2 = 0
        for it in dataset:
            if set(i).issubset(dataset[it]):
                numerator += 1
            if i[0] in dataset[it]:
                denominator1 += 1
            if i[1] in dataset[it]:
                denominator2 += 1
        cur_confidence = numerator / denominator1
        if cur_confidence >= min_confidence:
            print(f'{i[0]}->{i[1]} with confidence = {cur_confidence}')
        cur_confidence = numerator / denominator2
        if cur_confidence >= min_confidence:
            print(f'{i[1]}->{i[0]} with confidence = {cur_confidence}')


In [135]:
association(data=frequent_itemset, min_confidence=0.5)

Pastry->Coffee with confidence = 0.5521472392638037
Cake->Coffee with confidence = 0.5269582909460834
